In [2]:
from transformers import pipeline
from bs4 import BeautifulSoup
import requests

In [3]:
summarizer = pipeline("summarization")

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 (https://huggingface.co/sshleifer/distilbart-cnn-12-6)
2022-08-10 19:30:44.791960: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:975] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-08-10 19:30:44.825664: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudnn.so.8'; dlerror: libcudnn.so.8: cannot open shared object file: No such file or directory
2022-08-10 19:30:44.825681: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1850] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2022-08-10 19:30:44.826129: I tensorflow/co

In [17]:
# URL = "https://towardsdatascience.com/a-bayesian-take-on-model-regularization-9356116b6457"
# URL = "https://hackernoon.com/will-the-game-stop-with-gamestop-or-is-this-just-the-beginning-2j1x32aa"
URL = "https://ndrohith09.hashnode.dev/why-do-farmers-want-to-give-up-farming"

In [18]:
r = requests.get(URL)

In [19]:
soup = BeautifulSoup(r.text, 'html.parser')
results = soup.find_all(['h1', 'p'])
text = [result.text for result in results]
ARTICLE = ' '.join(text)

In [ ]:
ARTICLE

In [26]:
max_chunk = 500
ARTICLE = ARTICLE.replace('.', '.<eos>')
ARTICLE = ARTICLE.replace('?', '?<eos>')
ARTICLE = ARTICLE.replace('!', '!<eos>')

In [27]:
sentences = ARTICLE.split('<eos>')
current_chunk = 0 
chunks = []
for sentence in sentences:
    if len(chunks) == current_chunk + 1: 
        if len(chunks[current_chunk]) + len(sentence.split(' ')) <= max_chunk:
            chunks[current_chunk].extend(sentence.split(' '))
        else:
            current_chunk += 1
            chunks.append(sentence.split(' '))
    else:
        print(current_chunk)
        chunks.append(sentence.split(' '))

for chunk_id in range(len(chunks)):
    chunks[chunk_id] = ' '.join(chunks[chunk_id])

0


In [28]:
len(chunks)

3

In [29]:
res = summarizer(chunks, max_length=150, min_length=30, do_sample=False)
res

[{'summary_text': ' According to a poll of 5,000 farm households from 18 states, 76% of farmers want to give up farming . 61% of these farmers would prefer to work in cities due to better education, health care, and career opportunities . India exported agricultural goods worth almost 20674 million USD .'},
 {'summary_text': ' There is a significant geographical imbalance in rainfall and water supply in India . Agriculture, being one of the most significant sectors, need money . Lack of capital input is more vital for the growth of agricultural technology . Agricultural marketing in rural India remains poor .'},
 {'summary_text': ' GPS may be used for precision farming, in which satellites and drones collect data on plants and weather directly from the sky . Precision agriculture allows farmers to minimise the quantity of fertiliser put to the field . SMOS technology can forecast agricultural production characteristics such as floods, droughts, and so on .'}]

In [30]:
final_text = ' '.join([summ['summary_text'] for summ in res])
final_text 

' According to a poll of 5,000 farm households from 18 states, 76% of farmers want to give up farming . 61% of these farmers would prefer to work in cities due to better education, health care, and career opportunities . India exported agricultural goods worth almost 20674 million USD .  There is a significant geographical imbalance in rainfall and water supply in India . Agriculture, being one of the most significant sectors, need money . Lack of capital input is more vital for the growth of agricultural technology . Agricultural marketing in rural India remains poor .  GPS may be used for precision farming, in which satellites and drones collect data on plants and weather directly from the sky . Precision agriculture allows farmers to minimise the quantity of fertiliser put to the field . SMOS technology can forecast agricultural production characteristics such as floods, droughts, and so on .'